# Transfer Learning - Feature Extraction

In [1]:
# Importing TF and checking the version
import tensorflow as tf
print(tf.__version__)

2.10.0


In [2]:
# Inspecting our dataset
import os

for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in {dirpath}.")

There are 2 directories and 0 images in 10_food_classes_10_percent.
There are 10 directories and 0 images in 10_food_classes_10_percent\test.
There are 0 directories and 250 images in 10_food_classes_10_percent\test\chicken_curry.
There are 0 directories and 250 images in 10_food_classes_10_percent\test\chicken_wings.
There are 0 directories and 250 images in 10_food_classes_10_percent\test\fried_rice.
There are 0 directories and 250 images in 10_food_classes_10_percent\test\grilled_salmon.
There are 0 directories and 250 images in 10_food_classes_10_percent\test\hamburger.
There are 0 directories and 250 images in 10_food_classes_10_percent\test\ice_cream.
There are 0 directories and 250 images in 10_food_classes_10_percent\test\pizza.
There are 0 directories and 250 images in 10_food_classes_10_percent\test\ramen.
There are 0 directories and 250 images in 10_food_classes_10_percent\test\steak.
There are 0 directories and 250 images in 10_food_classes_10_percent\test\sushi.
There are 

In [3]:
# Creating data loaders
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 5

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images")
train_data_10_percent = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

print("Testing images")
test_data_10_percent = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

Training images
Found 750 images belonging to 10 classes.
Testing images
Found 2500 images belonging to 10 classes.


In [4]:
# Creating and using callbacks
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"Saving TensorBoard log files to {log_dir}")
    return tensorboard_callback

## Re-using trained models from TensorFlow Hub

In [5]:
# Getting trained image classification ML models from tfhub.dev

url_resnet = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"
url_efficientnet = "https://tfhub.dev/tensorflow/efficientnet/b0/feature_vector/1"

In [7]:
# Import dependencies
import tensorflow_hub as hub
from tensorflow.keras import layers

In [8]:
# Function to create a model from URL

def create_model(model_url, num_classes=10):
    
    # Download TFHub model and save as a Keras layer
    feature_extractor_layer = hub.KerasLayer(
        model_url,
        trainable=False,
        name="feature_extraction_layer",
        input_shape=IMAGE_SHAPE+(3,)
    )

    # Create our model
    model = tf.keras.Sequential([
        feature_extractor_layer,
        layers.Dense(num_classes, activation="softmax", name="output_layer")
    ])

    return model